In [22]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

from auxiliary import values as v
from auxiliary.utils.colors import bcolors as c
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from filtering import cardiac_region as cr
from filtering.run_filter_tissue import filter_by_tissue
from feature_extraction.feature_extractor import filter_by_margin, filter_by_volume
from auxiliary.data.dataset_ht import HtDataset
from nuclei_segmentation.run_cellpose import predict
import json


ds = HtDataset()

In [3]:
specimen = '0806_E5'
img_path = v.data_path + 'Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz'
img_path_gt = v.data_path + 'Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_GT.nii.gz'

img = imaging.read_image(img_path, axes='XYZ', verbose=1)
img_gt = imaging.read_image(img_path_gt, axes='XYZ', verbose=1)

Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/RawImages/Nuclei/QC_CROP/20190208_E2_DAPI_decon_0.5_crop.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_GT.nii.gz


In [10]:
tests = [
    '2D_5_6_45_M_AD',
    '2D_5_6_45_M_BI',
    '2D_5_6_45_M_AD_BI',
    '2D_5_6_45_M_EQ_AD_BI',
    '2D_5_6_45_M_BI_AD',
    '2D_5_6_45_M_EQ_BI_AD',
    '3D_5_6_45_M_AD',
    '3D_5_6_45_M_BI',
    '3D_5_6_45_M_AD_BI',
    '3D_5_6_45_M_EQ_AD_BI',
    '3D_5_6_45_M_BI_AD',
    '3D_5_6_45_M_EQ_BI_AD'
]

results = pd.read_csv('../nuclei_segmentation/results.csv')

In [11]:
results.head(12)

,Method,Pixel Accuracy,IoU,Mean IoU,Precision,Mean Precision,Recall,Mean Recall,F1-Score,Dice Coefficient,...,Mean Hausdorff Distance,NO_cells,VJI,ROS,RUS,JI_list,ROS_list,RUS_list,ROS_map,RUS_map
0,2D_5_6_45_M_AD,0.859877,0.696358,0.426008,0.958647,0.632786,0.717923,0.594904,0.821004,0.821004,...,10.130524,181,0.369788,0.751381,0.686170,"0.740506329113924,0.6595092024539877,0.6687780...","0,1,1,0,1,0,0,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,1,...","0,1,0,0,1,1,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [50.0], ""2.0"": [45.0, 51.0, 72.0, 103...."
1,2D_5_6_45_M_BI,0.850231,0.672040,0.449228,0.971335,0.713063,0.685638,0.582741,0.803857,0.803857,...,9.464205,209,0.352266,0.708134,0.712766,"0.717206132879046,0.6102403343782654,0.6587792...","0,1,1,0,1,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,...","0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [50.0], ""2.0"": [46.0, 51.0, 96.0, 121...."
2,2D_5_6_45_M_AD_BI,0.859429,0.696451,0.442156,0.954204,0.687012,0.720535,0.606296,0.821068,0.821068,...,9.504254,211,0.337795,0.734597,0.728723,"0.7580645161290323,0.6834319526627219,0.665226...","0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,0,0,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [53.0], ""2.0"": [48.0, 54.0, 86.0, 125...."
3,2D_5_6_45_M_EQ_AD_BI,0.812435,0.590812,0.362690,0.961754,0.625148,0.605027,0.491777,0.742780,0.742780,...,10.749163,172,0.312657,0.715116,0.569149,"0.7231222385861561,0.645,0.6595010615711253,0....","0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,...","1,1,1,1,1,1,1,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [47.0, 95.0], ""2.0"": [43.0, 48.0, 93.0..."
4,2D_5_6_45_M_BI_AD,0.853214,0.681690,0.424704,0.958743,0.641121,0.702292,0.584788,0.810720,0.810720,...,9.987728,191,0.349944,0.738220,0.691489,"0.7403846153846154,0.6646279306829765,0.667018...","0,1,1,0,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,...","0,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [51.0], ""2.0"": [46.0, 52.0, 74.0, 108...."
5,2D_5_6_45_M_EQ_BI_AD,0.812409,0.591054,0.372580,0.960657,0.614370,0.605716,0.488664,0.742971,0.742971,...,10.643526,163,0.336860,0.748466,0.526596,"0.7201166180758017,0.652834008097166,0.6361399...","0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,...","0,1,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,1,0,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [45.0], ""2.0"": [41.0, 46.0, 93.0, 96.0..."
6,3D_5_6_45_M_AD,0.852604,0.677626,0.400427,0.969924,0.525115,0.692170,0.595402,0.807839,0.807839,...,11.356249,108,0.459396,0.851852,0.558511,"0.7784810126582279,0.6775178026449644,0.630407...","0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,...","1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [54.0, 89.0], ""2.0"": [50.0, 55.0], ""3...."
7,3D_5_6_45_M_BI,0.855769,0.683367,0.413654,0.975251,0.546750,0.695427,0.593720,0.811905,0.811905,...,11.231549,111,0.471407,0.828829,0.595745,"0.7589134125636672,0.7070914696813977,0.625785...","0,1,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,...","1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,0,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [52.0, 91.0], ""2.0"": [49.0, 53.0], ""3...."
8,3D_5_6_45_M_AD_BI,0.863604,0.702771,0.400685,0.966207,0.504313,0.720480,0.616827,0.825444,0.825444,...,11.723671,104,0.469225,0.903846,0.579787,"0.7841269841269841,0.6708984375,0.631141530188...","0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [53.0, 87.0], ""2.0"": [49.0, 54.0], ""3...."
9,3D_5_6_45_M_EQ_AD_BI,0.761448,0.473282,0.315027,0.975920,0.560693,0.4788

In [12]:
metrics = results.columns[:-5]
cell_metrics = results.columns[-5:]

print(metrics)
print(cell_metrics)

Index(['Method', 'Pixel Accuracy', 'IoU', 'Mean IoU', 'Precision',
       'Mean Precision', 'Recall', 'Mean Recall', 'F1-Score',
       'Dice Coefficient', 'Mean Dice Coefficient', 'Hausdorff Distance',
       'Mean Hausdorff Distance', 'NO_cells', 'VJI', 'ROS', 'RUS'],
      dtype='object')
Index(['JI_list', 'ROS_list', 'RUS_list', 'ROS_map', 'RUS_map'], dtype='object')


In [14]:
results[metrics].describe()

,Pixel Accuracy,IoU,Mean IoU,Precision,Mean Precision,Recall,Mean Recall,F1-Score,Dice Coefficient,Mean Dice Coefficient,Hausdorff Distance,Mean Hausdorff Distance,NO_cells,VJI,ROS,RUS
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,0.832914,0.634660,0.393323,0.966599,0.594689,0.649545,0.547896,0.773318,0.773318,0.505911,15.277535,10.650179,149.000000,0.383926,0.778328,0.582890
std,0.037561,0.084384,0.044944,0.007657,0.067851,0.089092,0.076489,0.067359,0.067359,0.047246,2.561303,0.738567,42.784874,0.061239,0.066161,0.112367
min,0.760974,0.472020,0.312163,0.954204,0.504313,0.477402,0.405998,0.641322,0.641322,0.422491,10.198039,9.464205,104.000000,0.312657,0.708134,0.382979
25%,0.812429,0.590993,0.370107,0.960178,0.541341,0.605543,0.490998,0.742924,0.742924,0.485009,14.000000,10.094825,110.250000,0.341715,0.734931,0.550532
50%,0.852787,0.678038,0.400618,0.968038,0.591733,0.692614,0.589254,0.808132,0.808132,0.508144,14.844072,10.775320,140.000000,0.351105,0.749924,0.574468
75%,0.856684,0.686615,0.425030,0.972314,0.634870,0.706200,0.596227,0.814180,0.814180,0.540997,17.691806,11.262724,183.500000,0.459368,0.834585,0.687500
max,0.863604,0.702771,0.449228,0.976673,0.713063,0.720535,0.616827,0.825444,0.825444,0.569312,19.339080,11.723671,211.000000,0.471407,0.903846,0.728723


In [16]:
results[cell_metrics].head(12)

,JI_list,ROS_list,RUS_list,ROS_map,RUS_map
0,"0.740506329113924,0.6595092024539877,0.6687780...","0,1,1,0,1,0,0,1,1,0,1,0,1,1,1,1,0,1,1,1,1,1,1,...","0,1,0,0,1,1,0,1,1,1,1,1,0,1,0,0,1,1,1,1,1,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [50.0], ""2.0"": [45.0, 51.0, 72.0, 103...."
1,"0.717206132879046,0.6102403343782654,0.6587792...","0,1,1,0,1,0,1,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,...","0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [50.0], ""2.0"": [46.0, 51.0, 96.0, 121...."
2,"0.7580645161290323,0.6834319526627219,0.665226...","0,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,...","0,1,0,0,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [53.0], ""2.0"": [48.0, 54.0, 86.0, 125...."
3,"0.7231222385861561,0.645,0.6595010615711253,0....","0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,...","1,1,1,1,1,1,1,0,0,0,1,0,0,1,0,1,0,1,1,1,1,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [47.0, 95.0], ""2.0"": [43.0, 48.0, 93.0..."
4,"0.7403846153846154,0.6646279306829765,0.667018...","0,1,1,0,1,0,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,...","0,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [51.0], ""2.0"": [46.0, 52.0, 74.0, 108...."
5,"0.7201166180758017,0.652834008097166,0.6361399...","0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,...","0,1,0,0,1,1,1,0,0,0,1,0,0,1,0,0,0,1,1,1,1,1,0,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [45.0], ""2.0"": [41.0, 46.0, 93.0, 96.0..."
6,"0.7784810126582279,0.6775178026449644,0.630407...","0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,...","1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [54.0, 89.0], ""2.0"": [50.0, 55.0], ""3...."
7,"0.7589134125636672,0.7070914696813977,0.625785...","0,1,1,0,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,...","1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,0,1,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [52.0, 91.0], ""2.0"": [49.0, 53.0], ""3...."
8,"0.7841269841269841,0.6708984375,0.631141530188...","0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,...","1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,1,1,1,1,0,1,...","{""1.0"": [48.0], ""2.0"": [50.0, 52.0, 53.0], ""3....","{""1.0"": [53.0, 87.0], ""2.0"": [49.0, 54.0], ""3...."
9,"0.7302052785923754,0.8487115544472152,0.702127...","0,1,0,1,1,0,1,1,1,1,0,1,1,1,0,1,1,0,1,1,1,1,1,...","1,1,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,...","{""1.0"": [48.0], ""4.0"": [99.0, 176.0], ""5.0"": [...","{""1.0"": [52.0, 90.0], ""2.0"": [45.0, 51.0, 66.0..."


Compute and save ROS and RUS maps for each test

In [67]:
GREEN = [0, 255, 0]  # Correct prediction
RED = [255, 0, 0]    # Over-segmented
BLUE = [0, 0, 255]   # Under-segmented
WHITE = [255, 255, 255]  # Mistaken background
# GREEN = 2  # Correct prediction
# RED = 1    # Over-segmented
# BLUE = 3   # Under-segmented
# WHITE = 4  # Mistaken background


for test in tests:
    res = results[results['Method'] == test]
    
    rgb_image = np.zeros((img_gt.shape[0], img_gt.shape[1], img_gt.shape[2], 3), dtype=np.uint8)
    
    RUS_map = json.loads(res['RUS_map'].values[0])
    ROS_map = json.loads(res['ROS_map'].values[0])
        
    for gt_label, pred_cells in RUS_map.items():
        gt_label = int(float(gt_label))
        pred_cells = [int(float(cell)) for cell in pred_cells]
        
        if len(pred_cells) == 1:
            # Correctly predicted cells
            mask = np.where(img_gt == gt_label)
            rgb_image[mask] = GREEN
        elif len(pred_cells) > 1:
            # Over-segmented cells
            mask = np.where(img_gt == gt_label)
            rgb_image[mask] = RED
        elif len(pred_cells) == 0:
            # No prediction for this cell
            mask = np.where(img_gt == gt_label)
            rgb_image[mask] = WHITE
    
    for pred_label, gt_cells in ROS_map.items():
        pred_label = int(float(pred_label))
        gt_cells = [int(float(cell)) for cell in gt_cells]
        
        if len(gt_cells) == 1:
            # Correctly predicted cells (Already colored in the RUS loop)
            pass  # Skip because it's already marked
        elif len(gt_cells) > 1:
            # Under-segmented cells
            for cell in gt_cells:
                mask = np.where(img_gt == cell)
                rgb_image[mask] = BLUE
        elif len(gt_cells) == 0:
            # No ground truth for this cell
            pass
                
    # Save the RGB image
    imaging.save_tiff_imagej_compatible(
        v.data_path + f'Gr1/Segmentation/Nuclei/QC_CROP/ROS_RUS/{test}_ROS_RUS_map.tif',
        rgb_image,
        axes='XYZC',
    )